### This notebook contains code the final code of Le recommandeur. It can be used by the user for recommendation.

### Contributors: Developed by Gökalp Özer with the help of Mete Mert Birdal and Abdullah Palaz

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

import keras
from keras import models
from keras import layers

import os
from os import listdir
from os.path import isfile, join

In [ ]:
data = pd.read_csv('data.csv')

print("**************************************************************")
print("*                                                            *")
print("* Welcome To The Le Recommandeur Music Recommendation System *")
print("*                                                            *")
print("**************************************************************")

random_number = int(input("Please Enter a Random Number: "))
data = data.sample(frac = 1, random_state = random_number).reset_index(drop=True)
print("--------------------------------------------------------------")

print("Random 70 Songs For Rating:")
for i in range(70):
    print("-",data["song_name"][i])

print("--------------------------------------------------------------")

y = list(map(int, input("Please Rate The Given Songs as 0 and 1, Leaving a Space Between Each of Them(1 = I like it, 0 = I don't like it): ").split()))
y = np.array(y)

x = data.iloc[0:70, 1:]
X_test = data.iloc[70:, 1:]
#train-test split
X_train, validation_x, y_train, validation_y = train_test_split(x,y,test_size=0.25)

print("--------------------------------------------------------------")

print("Searching For The Most Suitable Songs For You According To The Ratings You Entered...")

# normalizing
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
validation_x = scaler.fit_transform(validation_x)
X_test = scaler.fit_transform(X_test)

model = models.Sequential()
model.add(layers.Dense(20, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train,
                    y_train,
                    epochs=45,
                    batch_size=4,
                    validation_data=(validation_x, validation_y))

print("--------------------------------------------------------------")

names = pd.DataFrame(data.iloc[70:, :1].values, columns = ["song_names"])
values = pd.DataFrame(model.predict(X_test), columns = ["values"])
recommend_df = pd.concat([names, values], axis=1).sort_values("values", ascending=False).reset_index(drop=True)

print("Recommended Songs:")
for i in range(10):
    print("-",recommend_df["song_names"][i])